##### Import package

In [ ]:
import numpy as np 
import pandas as pd 
import xarray as xr 
import cartopy 
import glob
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import scipy 

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline, make_union
from cartopy.util import add_cyclic_point
import warnings
import seaborn as sns
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.offsetbox as offsetbox
import seaborn.objects as so
from seaborn import axes_style
import matplotlib as mpl
import pickle
from matplotlib import cm

from matplotlib.ticker import MaxNLocator
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.gridspec as gridspec
import cartopy.feature as cfeature

from global_var_and_func import *
from plot_func import *


warnings.filterwarnings("ignore")
#from sklearn_xarray import Stacker, Select

##### Load Data

All Spatial plots are calculated with with `_addcyclic.pkl` for central longitude=180, where an additional point at 180 longitude is interpolated with cartopy addcyclic function.

In [ ]:
#Kernel Data
with open('kernel.pkl', 'rb') as f:
    (sst_kernels, eis_kernels, tadv_kernels, rh_kernels, omega_kernels, ws_kernels,
     sst_obs_kernels, eis_obs_kernels, tadv_obs_kernels, rh_obs_kernels, omega_obs_kernels, ws_obs_kernels) = pickle.load(f)

with open('data_smth/dCCF_dTg_smoothed_addcyclic.pkl', 'rb') as f:
    (sst_perglobts,eis_perglobts,tadv_perglobts,rh_perglobts,omega_perglobts,ws_perglobts)=pickle.load(f)

with open('data_smth/dR_tot_var_smth_addcyclic.pkl', 'rb') as f:
    dR_modk_modc_var,tot_kp_cm_var,tot_km_cp_var,var_diff = pickle.load(f)

with open('data_smth/dR_ccf_var_smth_addcyclic.pkl', 'rb') as f:
    var_dR_sst,var_dR_eis,var_dR_tadv,var_dR_rh,var_dR_omega,var_dR_ws = pickle.load(f)


In [ ]:
#  Radiative Feedbacks ccf-permodel*kernel-permod AND  ccf-permod * obs-kernel|MM-kernel 
dR_sst_perglobts,dR_sst_perglobts_ceres,dR_sst_perglobts_isccp,dR_sst_perglobts_modis,dR_sst_perglobts_patmosx,dR_sst_perglobts_mmkern = get_feedback(sst_perglobts.to_dataset(dim='model'),sst_kernels,obs_kernels=sst_obs_kernels)
dR_eis_perglobts,dR_eis_perglobts_ceres,dR_eis_perglobts_isccp,dR_eis_perglobts_modis,dR_eis_perglobts_patmosx,dR_eis_perglobts_mmkern = get_feedback(eis_perglobts.to_dataset(dim='model'),eis_kernels,obs_kernels=eis_obs_kernels)
dR_tadv_perglobts,dR_tadv_perglobts_ceres,dR_tadv_perglobts_isccp,dR_tadv_perglobts_modis,dR_tadv_perglobts_patmosx,dR_tadv_perglobts_mmkern = get_feedback(tadv_perglobts.to_dataset(dim='model'),tadv_kernels,obs_kernels=tadv_obs_kernels)
dR_rh_perglobts,dR_rh_perglobts_ceres,dR_rh_perglobts_isccp,dR_rh_perglobts_modis,dR_rh_perglobts_patmosx,dR_rh_perglobts_mmkern = get_feedback(rh_perglobts.to_dataset(dim='model'),rh_kernels,obs_kernels=rh_obs_kernels)
dR_omega_perglobts,dR_omega_perglobts_ceres,dR_omega_perglobts_isccp,dR_omega_perglobts_modis,dR_omega_perglobts_patmosx,dR_omega_perglobts_mmkern = get_feedback(omega_perglobts.to_dataset(dim='model'),omega_kernels,obs_kernels=omega_obs_kernels)
dR_ws_perglobts,dR_ws_perglobts_ceres,dR_ws_perglobts_isccp,dR_ws_perglobts_modis,dR_ws_perglobts_patmosx,dR_ws_perglobts_mmkern = get_feedback(ws_perglobts.to_dataset(dim='model'),ws_kernels,obs_kernels=ws_obs_kernels)

In [ ]:
# Calculate Ensemble Mean
names = ['sst', 'eis', 'tadv', 'rh', 'omega', 'ws']
var = [sst_perglobts, eis_perglobts, tadv_perglobts, rh_perglobts, omega_perglobts, ws_perglobts]
for i, name in enumerate(names):
    globals()[f"{name}_perglobts_hat"] = ensemble_mean(var[i])

#  Radiative Feedbacks [model, lat,lon,exp] MMM-ccf*kernel-permod - showing model kernel spread  
dR_sst_perglobts_mmccf = get_feedback(sst_perglobts_hat,sst_kernels).to_array(dim='model')
dR_eis_perglobts_mmccf = get_feedback(eis_perglobts_hat,eis_kernels).to_array(dim='model')
dR_tadv_perglobts_mmccf = get_feedback(tadv_perglobts_hat,tadv_kernels).to_array(dim='model')
dR_rh_perglobts_mmccf= get_feedback(rh_perglobts_hat,rh_kernels).to_array(dim='model')
dR_omega_perglobts_mmccf = get_feedback(omega_perglobts_hat,omega_kernels).to_array(dim='model')
dR_ws_perglobts_mmccf = get_feedback(ws_perglobts_hat,ws_kernels).to_array(dim='model')

In [ ]:
# dR [model,exp,lat,lon] - with 6+1 ccfs (from ccf-permod * kernel-permod) 
dR_modelccf_modelkern = ccf_feedbacks(dR_sst_perglobts,dR_eis_perglobts,dR_tadv_perglobts,dR_rh_perglobts,dR_omega_perglobts,dR_ws_perglobts)
dR_modelccf_cereskern = ccf_feedbacks(dR_sst_perglobts_ceres,dR_eis_perglobts_ceres,dR_tadv_perglobts_ceres,dR_rh_perglobts_ceres,dR_omega_perglobts_ceres,dR_ws_perglobts_ceres)
dR_modelccf_isccpkern = ccf_feedbacks(dR_sst_perglobts_isccp,dR_eis_perglobts_isccp,dR_tadv_perglobts_isccp,dR_rh_perglobts_isccp,dR_omega_perglobts_isccp,dR_ws_perglobts_isccp)
dR_modelccf_modiskern = ccf_feedbacks(dR_sst_perglobts_modis,dR_eis_perglobts_modis,dR_tadv_perglobts_modis,dR_rh_perglobts_modis,dR_omega_perglobts_modis,dR_ws_perglobts_modis)
dR_modelccf_patmoskern = ccf_feedbacks(dR_sst_perglobts_patmosx,dR_eis_perglobts_patmosx,dR_tadv_perglobts_patmosx,dR_rh_perglobts_patmosx,dR_omega_perglobts_patmosx,dR_ws_perglobts_patmosx)

dR_modelccf_mmkern = ccf_feedbacks(dR_sst_perglobts_mmkern,dR_eis_perglobts_mmkern,dR_tadv_perglobts_mmkern,dR_rh_perglobts_mmkern,dR_omega_perglobts_mmkern,dR_ws_perglobts_mmkern)
dR_mmccf_modelkern = ccf_feedbacks(dR_sst_perglobts_mmccf,dR_eis_perglobts_mmccf,dR_tadv_perglobts_mmccf,dR_rh_perglobts_mmccf,dR_omega_perglobts_mmccf,dR_ws_perglobts_mmccf)


## MMM of feedback (from ccf-permod * kernel-permod) (Spatial Maps)
dR_modelccf_modelkern_hat, dR_modelccf_modelkern_hat_zonal = hat_and_zonal(dR_modelccf_modelkern)

#### Fig 2 dCCF

In [ ]:
# Main Fig 2
test = False
levels = [np.arange(-2,2+0.1,0.1),
         np.arange(-1.5,1.5+0.1,0.1)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.TwoSlopeNorm(vcenter=0, vmin=-2, vmax=2.0),\
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-1.5, vmax=1.5)] #vcenter=cbarcent, vmin=minval, vmax=maxval


ds = [sst_perglobts_hat, eis_perglobts_hat]
units_type =  'ccf' #or 'dR'
leg_loc = None#'upper right'
row_titles = ['$\mathbf{dSST/dT_{g}}$', '$\mathbf{dEIS/dT_{g}}$']


# Plotting Figures =========================================================================================================
row = 2
col = 5

fig = plt.figure(figsize=(28,7), dpi=250) #28,7
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.34, hspace=0.27)

add_row_titles_from_gridspec(fig, row_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):
    for j in range(0,col): 
        if units_type == 'ccf':
            ylabel = 'K/K'
        elif units_type == 'dR':
            ylabel='$Wm^{-2}K^{-1}$' 

        if j == col-1: #zonal mean plots 
            ax[i][j] = fig.add_subplot(gs[i, j])
            zonal = ds[i].mean(dim='lon')
            for k in range(4):
                ax[i][j].plot(zonal.lat, zonal.isel(exp=k), zonal_plt_color[k], label=ds[i].exp.values[k])
            
            ax[i][j].set_xlabel('Latitude (\N{DEGREE SIGN})', fontsize=15)
            ax[i][j].yaxis.set_label_position("right")
            ax[i][j].yaxis.tick_right()
            ax[i][j].tick_params(axis='y', labelsize=15)
            ax[i][j].set_ylabel(ylabel, fontsize=15)
            ax[i][j].grid(True)
            
            make_textbox(ax[i][j],textbox_labels[col*i+j]) # required to be called after gs objects are made 
            if i == 0:
                ax[i][j].set_title(column_titles[j], fontsize=18,fontweight='bold')
            if i == 1: 
                ax[i][j].legend(loc=leg_loc)

        
        else: #maps 
            if test == True: 
                ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
                map = ax[i][j].contourf(ds[i].lon,ds[i].lat,(ds[i].sel(exp=ds[i].exp.values[j])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
                print('test plotting subplot',col*i+j)
            
            elif test == False:     
                ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
                map = ax[i][j].contourf(ds[i].lon,ds[i].lat,(ds[i].sel(exp=ds[i].exp.values[j])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[i],levels=levels[i])
                print('plotting subplot',col*i+j)

            make_textbox(ax[i][j],textbox_labels[col*i+j])
            ax[i][j].coastlines()
            ax[i][j].set_aspect('auto')
            ax[i][j].set_global()
            ax[i][j].add_feature(cfeature.LAND, zorder=3)
            
            ax[i][j].set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree(central_longitude=180))
            gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--',
                                   xlocs=[-180, -120, -60, 0, 60, 120, 180])
            
            gl.top_labels = False
            gl.right_labels = False
            gl.xlabel_style = {'size': 14}
            gl.ylabel_style = {'size': 14}
            if i == 0:
                ax[i][j].set_title(column_titles[j], fontsize=18,fontweight='bold')
            
            if ax[i][3] is not None:
                cax = inset_axes(ax[i][3], width="5%", height="100%", loc='center left',
                     bbox_to_anchor=(1.05, 0, 1, 1),
                     bbox_transform=ax[i][3].transAxes, borderpad=0)
                cbar = fig.colorbar(map, cax=cax, orientation='vertical')#,label=ylabel)
                cbar.ax.tick_params(labelsize=15)

plt.savefig('dCCF_main_2x5map_smth.png',  dpi=400, bbox_inches='tight')

In [ ]:
# Supp Fig 2
test = False
levels = [np.arange(-0.4,0.4+0.05,0.05),
          np.arange(-7,7+1,1),
          np.arange(-9,9+1,1),
         np.arange(-0.7,0.7+0.1,0.1)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.TwoSlopeNorm(vcenter=0, vmin=-0.4, vmax=0.4),
             mcolors.TwoSlopeNorm(vcenter=0, vmin=-7, vmax=7),
             mcolors.TwoSlopeNorm(vcenter=0, vmin=-9, vmax=9),
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-0.7, vmax=0.7)] #vcenter=cbarcent, vmin=minval, vmax=maxval

ds = [tadv_perglobts_hat, rh_perglobts_hat, omega_perglobts_hat, ws_perglobts_hat]
units_type =  'ccf' #or 'dR'
leg_loc = None#'upper right'
row_titles = ['$\mathbf{dT_{adv}/dT_{g}}$', '$\mathbf{dRH/dT_{g}}$', '$\mathbf{d\omega/dT_{g}}$', '$\mathbf{dWS/dT_{g}}$']


# Plotting Figures =========================================================================================================

row = 4
col = 5

fig = plt.figure(figsize=(28, 14), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.32, hspace=0.27)

add_row_titles_from_gridspec(fig, row_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):
    for j in range(0,col): 
        if units_type == 'ccf':
            ylabel = ['$Kday^{-1}K^{-1}$','$K^{-1}$','$hPaDay^{-1}K^{-1}$','$ms^{-1}K^{-1}$']
        elif units_type == 'dR':
            ylabel='$Wm^{-2}K^{-1}$' 

        if j == col-1: #zonal mean
            ax[i][j] = fig.add_subplot(gs[i, j])
            zonal = ds[i].mean(dim='lon')
            for k in range(4):
                ax[i][j].plot(zonal.lat, zonal.isel(exp=k), zonal_plt_color[k], label=ds[i].exp.values[k])
            
            ax[i][j].set_xlabel('Latitude (\N{DEGREE SIGN})', fontsize=15)
            ax[i][j].yaxis.set_label_position("right")
            ax[i][j].yaxis.tick_right()
            ax[i][j].tick_params(axis='both', labelsize=15)
            ax[i][j].set_ylabel(ylabel[i], fontsize=16, labelpad=18.5)
            ax[i][j].grid(True)
            
            make_textbox(ax[i][j],textbox_labels[col*i+j]) # required to be called after gs objects are made 
            if i == 0:
                ax[i][j].set_title(column_titles[j], fontsize=18,fontweight='bold')
            if i == 1: 
                ax[i][j].legend(loc=leg_loc)
        
        else: #spatial maps 
            if test == True: 
                ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
                map = ax[i][j].contourf(ds[i].lon,ds[i].lat,(ds[i].sel(exp=ds[i].exp.values[j])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
                print('test plotting subplot',col*i+j)
            
            elif test == False:     
                ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
                map = ax[i][j].contourf(ds[i].lon,ds[i].lat,(ds[i].sel(exp=ds[i].exp.values[j])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[i],levels=levels[i])
                print('plotting subplot',col*i+j)

            make_textbox(ax[i][j],textbox_labels[col*i+j])
            ax[i][j].coastlines()
            ax[i][j].set_aspect('auto')
            ax[i][j].set_global()
            ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
            
            ax[i][j].set_extent([-180, 180, -61, 60], crs=ccrs.PlateCarree())
            gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
            gl.top_labels = False
            gl.right_labels = False
            gl.xlabel_style = {'size': 14}
            gl.ylabel_style = {'size': 14}
            if i == 0:
                ax[i][j].set_title(column_titles[j], fontsize=18,fontweight='bold')
            
            if ax[i][3] is not None:
                cax = inset_axes(ax[i][3], width="5%", height="100%", loc='center left',
                     bbox_to_anchor=(1.05, 0, 1, 1),
                     bbox_transform=ax[i][3].transAxes, borderpad=0)
                cbar = fig.colorbar(map, cax=cax, orientation='vertical')
                cbar.ax.tick_params(labelsize=14)
                cbar.ax.yaxis.set_major_locator(MaxNLocator(nbins=6)) 
                # cbar.set_label(ylabel[i], fontsize=15)

plt.savefig('dCCF_supp_4x5map_smth.png',  dpi=400, bbox_inches='tight')

#### Fig 3 dR

In [ ]:
from cartopy.util import add_cyclic_point 
dR_sst = dR_add_cyclic_point(dR_modelccf_modelkern_hat['sst'])
dR_eis = dR_add_cyclic_point(dR_modelccf_modelkern_hat['eis'])
dR_tadv = dR_add_cyclic_point(dR_modelccf_modelkern_hat['tadv'])
dR_rh = dR_add_cyclic_point(dR_modelccf_modelkern_hat['rh'])
dR_omega = dR_add_cyclic_point(dR_modelccf_modelkern_hat['omega'])
dR_ws = dR_add_cyclic_point(dR_modelccf_modelkern_hat['ws'])

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.ticker as tck

# Main Fig 3 
test = False
levels = [np.arange(-1.7,1.7+0.15,0.15),
         np.arange(-1.7,1.7+0.15,0.15)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.TwoSlopeNorm(vcenter=0, vmin=-1.7, vmax=1.7),\
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-1.7, vmax=1.7)] #vcenter=cbarcent, vmin=minval, vmax=maxval


ds = [dR_sst, dR_eis]
ymin = -1.5 #from dR_eis
ymax = 1.3 #from dR_sst
units_type =  'dR' #or 'ccf'
leg_loc = None#'upper right'
row_titles = ['$\mathbf{dR_{SST}/dT_{g}}$', '$\mathbf{dR_{EIS}/dT_{g}}$']


# Plotting Figures =========================================================================================================
row = 2
col = 5

fig = plt.figure(figsize=(28, 7), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.33, hspace=0.27)

add_row_titles_from_gridspec(fig, row_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):
    for j in range(0,col): 
        if units_type == 'ccf':
            ylabel = 'K/K'
        elif units_type == 'dR':
            ylabel='$Wm^{-2}K^{-1}$' 

        if j == col-1:
            ax[i][j] = fig.add_subplot(gs[i, j])
            zonal = ds[i].mean(dim='lon')
            for k in range(4):
                ax[i][j].plot(zonal.lat, zonal.isel(exp=k), zonal_plt_color[k], label=ds[i].exp.values[k])
            ax[i][j].set_xlabel('Latitude (\N{DEGREE SIGN})', fontsize=15)
            ax[i][j].set_ylim([ymin, ymax]) 
            ax[i][j].yaxis.set_minor_locator(tck.AutoMinorLocator(2))
            ax[i][j].grid(True, which='minor' ,color='lightgray')
            ax[i][j].grid(True)
            ax[i][j].yaxis.set_label_position("right")
            ax[i][j].yaxis.tick_right()
            ax[i][j].tick_params(axis='y', labelsize=15)
            ax[i][j].set_ylabel(ylabel, fontsize=16, labelpad=18)
            
            make_textbox(ax[i][j],textbox_labels[col*i+j])
            if i == 0:
                ax[i][j].set_title(column_titles[j], fontsize=18,fontweight='bold')
            if i == 1:
                ax[i][j].legend(loc=leg_loc,ncol=2)
        else:
            if test == True: 
                ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
                map = ax[i][j].contourf(ds[i].lon,ds[i].lat,(ds[i].sel(exp=ds[i].exp.values[j])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
                print('test plotting subplot',col*i+j)
            
            elif test == False:     
                ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
                map = ax[i][j].contourf(ds[i].lon,ds[i].lat,(ds[i].sel(exp=ds[i].exp.values[j])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[i],levels=levels[i])
                print('plotting subplot',col*i+j)

            make_textbox(ax[i][j],textbox_labels[col*i+j])
            ax[i][j].coastlines()
            ax[i][j].set_aspect('auto')
            ax[i][j].set_global()
            ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
            
            ax[i][j].set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())
            gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
            gl.top_labels = False
            gl.right_labels = False
            gl.xlabel_style = {'size': 14}
            gl.ylabel_style = {'size': 14}
            if i == 0:
                ax[i][j].set_title(column_titles[j], fontsize=18,fontweight='bold')
                
        cbar_ax = fig.add_axes([0.742, 0.125, 0.008, 0.74])  # [left, bottom, width, height]
        cbar = fig.colorbar(map, cax=cbar_ax, orientation='vertical')
        cbar.ax.tick_params(labelsize=14)
        # cbar.ax.yaxis.set_major_locator(MaxNLocator(nbins=6)) 
        # cbar.set_label(label=ylabel, fontsize=15)

plt.savefig('dR_main_2x5map_smth.png',  dpi=400, bbox_inches='tight')

In [ ]:
# Supp Fig 3
test = False

levels = np.arange(-1.,1.+0.15,0.15)
normalize = mcolors.TwoSlopeNorm(vcenter=0, vmin=-1., vmax=1.)          

ds = [dR_tadv, dR_rh, dR_omega, dR_ws]
ymin = -1.5 #from dR_eis
ymax = 1.3 #from dR_sst
units_type =  'dR' #or 'ccf'
leg_loc = None#'upper right'
row_titles = ['$\mathbf{dR_{adv}/dT_{g}}$', '$\mathbf{dR_{RH}/dT_{g}}$', '$\mathbf{dR_{\omega}/dT_{g}}$', '$\mathbf{dR_{WS}/dT_{g}}$']

# Plotting Figures =========================================================================================================
row = 4
col = 5

fig = plt.figure(figsize=(28, 14), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.32, hspace=0.27)

add_row_titles_from_gridspec(fig, row_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):
    for j in range(0,col): 
        if units_type == 'ccf':
            ylabel = ['$Kday^{-1}K^{-1}$','$K^{-1}$','$hPaDay^{-1}K^{-1}$','$ms^{-1}K^{-1}$']
        elif units_type == 'dR':
            ylabel='$Wm^{-2}K^{-1}$' 

        if j == col-1:
            ax[i][j] = fig.add_subplot(gs[i, j])
            zonal = ds[i].mean(dim='lon')
            for k in range(4):
                ax[i][j].plot(zonal.lat, zonal.isel(exp=k), zonal_plt_color[k], label=ds[i].exp.values[k])
            ax[i][j].set_xlabel('Latitude (\N{DEGREE SIGN})', fontsize=15)
            ax[i][j].grid(True)
            ax[i][j].set_ylim([ymin, ymax]) 
            ax[i][j].yaxis.set_minor_locator(tck.AutoMinorLocator(2))
            ax[i][j].grid(True, which='minor' ,color='lightgray')
            ax[i][j].yaxis.set_label_position("right")
            ax[i][j].yaxis.tick_right()
            ax[i][j].tick_params(axis='y', labelsize=15)
            ax[i][j].set_ylabel(ylabel, fontsize=16,labelpad =18.5)
            make_textbox(ax[i][j],textbox_labels[col*i+j])
            if i == 0:
                ax[i][j].set_title(column_titles[j], fontsize=18,fontweight='bold')
            if i == 1:
                ax[i][j].legend(loc=leg_loc)
        else:
            if test == True: 
                ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
                map = ax[i][j].contourf(ds[i].lon,ds[i].lat,(ds[i].sel(exp=ds[i].exp.values[j])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
                print('test plotting subplot',col*i+j)
            
            elif test == False:     
                ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
                map = ax[i][j].contourf(ds[i].lon,ds[i].lat,(ds[i].sel(exp=ds[i].exp.values[j])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',levels=levels,norm=normalize)
                print('plotting subplot',col*i+j)

            make_textbox(ax[i][j],textbox_labels[col*i+j])
            ax[i][j].coastlines()
            ax[i][j].set_aspect('auto')
            ax[i][j].set_global()
            ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
            
            ax[i][j].set_extent([-180, 180, -61, 60], crs=ccrs.PlateCarree())
            gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
            gl.top_labels = False
            gl.right_labels = False
            gl.xlabel_style = {'size': 14}
            gl.ylabel_style = {'size': 14}
            if i == 0:
                ax[i][j].set_title(column_titles[j], fontsize=18,fontweight='bold')
            
        cbar_ax = fig.add_axes([0.742, 0.125, 0.008, 0.76])  # [left, bottom, width, height]
        cbar = fig.colorbar(map, cax=cbar_ax, orientation='vertical')
        cbar.ax.tick_params(labelsize=14)

plt.savefig('dR_supp_4x5map_smth.png',  dpi=400, bbox_inches='tight')

#### Fig 4 Var

##### Main Fig 4

In [ ]:
# Main Fig 4
test = False 
levels = [np.arange(0,13+0.5,0.5),
         np.arange(-7,7+0.5,0.5)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.Normalize(vmin=0, vmax=13),
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-7, vmax=7)] #vcenter=cbarcent, vmin=minval, vmax=maxval

ds = [dR_add_cyclic_point(dR_modk_modc_var), 
      dR_add_cyclic_point(tot_kp_cm_var), 
      dR_add_cyclic_point(tot_km_cp_var), 
      dR_add_cyclic_point(var_diff)]
units_type =  'dR' #or 'ccf'
leg_loc = None#'upper right'
row_titles = [
    r"$\mathbf{Var(dR)}$",
    r"$\mathbf{Var\left(\left(\frac{dR}{dX}\right)'\,\overline{dX}\right)}$",
    r"$\mathbf{Var\left(\overline{\frac{dR}{dX}}\,dX'\right)}$",
    r"$\mathbf{a - (b + c)}$"
]

col_titles = column_titles[:4]

# Plotting Figures =========================================================================================================
row = 4
col = 4

fig = plt.figure(figsize=(24, 14), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.32, hspace=0.27)

add_row_titles_from_gridspec(fig, col_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):#ds
    for j in range(0,col): #exp
        if units_type == 'ccf':
            ylabel = ['$Kday^{-1}K^{-1}$','$K^{-1}$','$hPaDay^{-1}K^{-1}$','$ms^{-1}K^{-1}$']
        elif units_type == 'dR':
            ylabel='$(Wm^{-2}K^{-1})^{2}$' 
   
        if test == True: 
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max')#,norm=normalize,levels=levels)
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
            print('test plotting subplot',col*i+j)
        
        elif test == False:     
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max',norm=normalize[0],levels=levels[0])
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[1],levels=levels[1])
            print('plotting subplot',col*i+j)

        make_textbox(ax[i][j],textbox_labels[col*i+j])
        ax[i][j].coastlines()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_global()
        ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
        
        ax[i][j].set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())
        gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        if j == 3:
            gl.left_labels = False
        gl.xlabel_style = {'size': 12}
        gl.ylabel_style = {'size': 12}
        if i == 0:
            ax[i][j].set_title(row_titles[j], fontsize=18,fontweight='bold')

    # var cbar
    cbar_ax = fig.add_axes([0.7, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    cbar = fig.colorbar(map, cax=cbar_ax, orientation='vertical')
    # cbar.set_label(label=ylabel, fontsize=16)
    cbar.ax.tick_params(labelsize=16)

    # covar cbar
    covbar_ax = fig.add_axes([0.91, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    covbar = fig.colorbar(covar_map, cax=covbar_ax, orientation='vertical')
    covbar.set_label(label=ylabel, fontsize=16)
    covbar.ax.tick_params(labelsize=16)
        
plt.savefig('vardR_main_4x4map_smth.png',  dpi=400, bbox_inches='tight')

##### Supp Figs

###### SST

In [ ]:
# Supp Fig 4 - SST 
test = False 
levels = [np.arange(0,13+0.5,0.5),
         np.arange(-7,7+0.5,0.5)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.Normalize(vmin=0, vmax=13),
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-7, vmax=7)] #vcenter=cbarcent, vmin=minval, vmax=maxval

ds = [dR_add_cyclic_point(var_dR_sst.sel(comb='ik_ic')), 
      dR_add_cyclic_point(var_dR_sst.sel(comb='kp_cm')), 
      dR_add_cyclic_point(var_dR_sst.sel(comb='km_cp')), 
      dR_add_cyclic_point(var_dR_sst.sel(comb='indv-aprx'))]
units_type =  'dR' #or 'ccf'
leg_loc = None#'upper right'
row_titles = [
    r"$\mathbf{Var(dR)}$",
    r"$\mathbf{Var\left(\left(\frac{dR}{dX}\right)'\,\overline{dX}\right)}$",
    r"$\mathbf{Var\left(\overline{\frac{dR}{dX}}\,dX'\right)}$",
    r"$\mathbf{a - (b + c)}$"
]

col_titles = column_titles[:4]

# Plotting Figures =========================================================================================================
row = 4
col = 4

fig = plt.figure(figsize=(24, 14), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.32, hspace=0.27)

add_row_titles_from_gridspec(fig, col_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):#ds
    for j in range(0,col): #exp
        if units_type == 'ccf':
            ylabel = ['$Kday^{-1}K^{-1}$','$K^{-1}$','$hPaDay^{-1}K^{-1}$','$ms^{-1}K^{-1}$']
        elif units_type == 'dR':
            ylabel='$(Wm^{-2}K^{-1})^{2}$'
   
        if test == True: 
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max')#,norm=normalize,levels=levels)
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
            print('test plotting subplot',col*i+j)
        
        elif test == False:     
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max',norm=normalize[0],levels=levels[0])
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[1],levels=levels[1])
            print('plotting subplot',col*i+j)

        make_textbox(ax[i][j],textbox_labels[col*i+j])
        ax[i][j].coastlines()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_global()
        ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
        
        ax[i][j].set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())
        gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        if j == 3:
            gl.left_labels = False
        gl.xlabel_style = {'size': 12}
        gl.ylabel_style = {'size': 12}
        if i == 0:
            ax[i][j].set_title(row_titles[j], fontsize=18,fontweight='bold')

    # var cbar
    cbar_ax = fig.add_axes([0.7, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    cbar = fig.colorbar(map, cax=cbar_ax, orientation='vertical')
    # cbar.set_label(label=ylabel, fontsize=16)
    cbar.ax.tick_params(labelsize=16)

    # covar cbar
    covbar_ax = fig.add_axes([0.91, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    covbar = fig.colorbar(covar_map, cax=covbar_ax, orientation='vertical')
    covbar.set_label(label=ylabel, fontsize=16)
    covbar.ax.tick_params(labelsize=16)
        
plt.savefig('vardR_supp_sst_4x4map_smth.png',  dpi=400, bbox_inches='tight')

###### EIS

In [ ]:
# Supp Fig 4 - EIS
test = False 
levels = [np.arange(0,13+0.5,0.5),
         np.arange(-7,7+0.5,0.5)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.Normalize(vmin=0, vmax=13),
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-7, vmax=7)] #vcenter=cbarcent, vmin=minval, vmax=maxval

ds = [dR_add_cyclic_point(var_dR_eis.sel(comb='ik_ic')), 
      dR_add_cyclic_point(var_dR_eis.sel(comb='kp_cm')), 
      dR_add_cyclic_point(var_dR_eis.sel(comb='km_cp')), 
      dR_add_cyclic_point(var_dR_eis.sel(comb='indv-aprx'))]
units_type =  'dR' #or 'ccf'
leg_loc = None#'upper right'
row_titles = [
    r"$\mathbf{Var(dR)}$",
    r"$\mathbf{Var\left(\left(\frac{dR}{dX}\right)'\,\overline{dX}\right)}$",
    r"$\mathbf{Var\left(\overline{\frac{dR}{dX}}\,dX'\right)}$",
    r"$\mathbf{a - (b + c)}$"
]

col_titles = column_titles[:4]

# Plotting Figures =========================================================================================================
row = 4
col = 4

fig = plt.figure(figsize=(24, 14), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.32, hspace=0.27)

add_row_titles_from_gridspec(fig, col_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):#ds
    for j in range(0,col): #exp
        if units_type == 'ccf':
            ylabel = ['$Kday^{-1}K^{-1}$','$K^{-1}$','$hPaDay^{-1}K^{-1}$','$ms^{-1}K^{-1}$']
        elif units_type == 'dR':
            ylabel='$(Wm^{-2}K^{-1})^{2}$'
   
        if test == True: 
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max')#,norm=normalize,levels=levels)
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
            print('test plotting subplot',col*i+j)
        
        elif test == False:     
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max',norm=normalize[0],levels=levels[0])
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[1],levels=levels[1])
            print('plotting subplot',col*i+j)

        make_textbox(ax[i][j],textbox_labels[col*i+j])
        ax[i][j].coastlines()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_global()
        ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
        
        ax[i][j].set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())
        gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        if j == 3:
            gl.left_labels = False
        gl.xlabel_style = {'size': 12}
        gl.ylabel_style = {'size': 12}
        if i == 0:
            ax[i][j].set_title(row_titles[j], fontsize=18,fontweight='bold')

    # var cbar
    cbar_ax = fig.add_axes([0.7, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    cbar = fig.colorbar(map, cax=cbar_ax, orientation='vertical')
    # cbar.set_label(label=ylabel, fontsize=16)
    cbar.ax.tick_params(labelsize=16)

    # covar cbar
    covbar_ax = fig.add_axes([0.91, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    covbar = fig.colorbar(covar_map, cax=covbar_ax, orientation='vertical')
    covbar.set_label(label=ylabel, fontsize=16)
    covbar.ax.tick_params(labelsize=16)
        
plt.savefig('vardR_supp_eis_4x4map_smth.png',  dpi=400, bbox_inches='tight')

###### Tadv

In [ ]:
# Supp Fig 4 - Tadv
test = False 
levels = [np.arange(0,7+0.25,0.25),
         np.arange(-2,3+0.25,0.25)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.Normalize(vmin=0, vmax=7),
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-2, vmax=3)] #vcenter=cbarcent, vmin=minval, vmax=maxval

ds = [dR_add_cyclic_point(var_dR_tadv.sel(comb='ik_ic')), 
      dR_add_cyclic_point(var_dR_tadv.sel(comb='kp_cm')), 
      dR_add_cyclic_point(var_dR_tadv.sel(comb='km_cp')), 
      dR_add_cyclic_point(var_dR_tadv.sel(comb='indv-aprx'))]
units_type =  'dR' #or 'ccf'
leg_loc = None#'upper right'
row_titles = [
    r"$\mathbf{Var(dR)}$",
    r"$\mathbf{Var\left(\left(\frac{dR}{dX}\right)'\,\overline{dX}\right)}$",
    r"$\mathbf{Var\left(\overline{\frac{dR}{dX}}\,dX'\right)}$",
    r"$\mathbf{a - (b + c)}$"
]

col_titles = column_titles[:4]

# Plotting Figures =========================================================================================================
row = 4
col = 4

fig = plt.figure(figsize=(24, 14), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.32, hspace=0.27)

add_row_titles_from_gridspec(fig, col_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):#ds
    for j in range(0,col): #exp
        if units_type == 'ccf':
            ylabel = ['$Kday^{-1}K^{-1}$','$K^{-1}$','$hPaDay^{-1}K^{-1}$','$ms^{-1}K^{-1}$']
        elif units_type == 'dR':
            ylabel='$(Wm^{-2}K^{-1})^{2}$'
   
        if test == True: 
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max')#,norm=normalize,levels=levels)
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
            print('test plotting subplot',col*i+j)
        
        elif test == False:     
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max',norm=normalize[0],levels=levels[0])
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[1],levels=levels[1])
            print('plotting subplot',col*i+j)

        make_textbox(ax[i][j],textbox_labels[col*i+j])
        ax[i][j].coastlines()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_global()
        ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
        
        ax[i][j].set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())
        gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        if j == 3:
            gl.left_labels = False
        gl.xlabel_style = {'size': 12}
        gl.ylabel_style = {'size': 12}
        if i == 0:
            ax[i][j].set_title(row_titles[j], fontsize=18,fontweight='bold')

    # var cbar
    cbar_ax = fig.add_axes([0.7, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    cbar = fig.colorbar(map, cax=cbar_ax, orientation='vertical')
    # cbar.set_label(label=ylabel, fontsize=16)
    cbar.ax.tick_params(labelsize=16)

    # covar cbar
    covbar_ax = fig.add_axes([0.91, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    covbar = fig.colorbar(covar_map, cax=covbar_ax, orientation='vertical')
    covbar.set_label(label=ylabel, fontsize=16)
    covbar.ax.tick_params(labelsize=16)
        
plt.savefig('vardR_supp_tadv_4x4map_smth.png',  dpi=400, bbox_inches='tight')

###### RH

In [ ]:
# Supp Fig 4 - RH
test = False 
levels = [np.arange(0,7+0.5,0.25),
         np.arange(-2,3+0.25,0.25)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.Normalize(vmin=0, vmax=7),
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-2, vmax=3)] #vcenter=cbarcent, vmin=minval, vmax=maxval

ds = [dR_add_cyclic_point(var_dR_rh.sel(comb='ik_ic')), 
      dR_add_cyclic_point(var_dR_rh.sel(comb='kp_cm')), 
      dR_add_cyclic_point(var_dR_rh.sel(comb='km_cp')), 
      dR_add_cyclic_point(var_dR_rh.sel(comb='indv-aprx'))]
units_type =  'dR' #or 'ccf'
leg_loc = None#'upper right'
row_titles = [
    r"$\mathbf{Var(dR)}$",
    r"$\mathbf{Var\left(\left(\frac{dR}{dX}\right)'\,\overline{dX}\right)}$",
    r"$\mathbf{Var\left(\overline{\frac{dR}{dX}}\,dX'\right)}$",
    r"$\mathbf{a - (b + c)}$"
]

col_titles = column_titles[:4]

# Plotting Figures =========================================================================================================
row = 4
col = 4

fig = plt.figure(figsize=(24, 14), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.32, hspace=0.27)

add_row_titles_from_gridspec(fig, col_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):#ds
    for j in range(0,col): #exp
        if units_type == 'ccf':
            ylabel = ['$Kday^{-1}K^{-1}$','$K^{-1}$','$hPaDay^{-1}K^{-1}$','$ms^{-1}K^{-1}$']
        elif units_type == 'dR':
            ylabel='$(Wm^{-2}K^{-1})^{2}$'
   
        if test == True: 
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max')#,norm=normalize,levels=levels)
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
            print('test plotting subplot',col*i+j)
        
        elif test == False:     
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max',norm=normalize[0],levels=levels[0])
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[1],levels=levels[1])
            print('plotting subplot',col*i+j)

        make_textbox(ax[i][j],textbox_labels[col*i+j])
        ax[i][j].coastlines()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_global()
        ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
        
        ax[i][j].set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())
        gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        if j == 3:
            gl.left_labels = False
        gl.xlabel_style = {'size': 12}
        gl.ylabel_style = {'size': 12}
        if i == 0:
            ax[i][j].set_title(row_titles[j], fontsize=18,fontweight='bold')

    # var cbar
    cbar_ax = fig.add_axes([0.7, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    cbar = fig.colorbar(map, cax=cbar_ax, orientation='vertical')
    # cbar.set_label(label=ylabel, fontsize=16)
    cbar.ax.tick_params(labelsize=16)

    # covar cbar
    covbar_ax = fig.add_axes([0.91, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    covbar = fig.colorbar(covar_map, cax=covbar_ax, orientation='vertical')
    covbar.set_label(label=ylabel, fontsize=16)
    covbar.ax.tick_params(labelsize=16)
    
plt.savefig('vardR_supp_rh_4x4map_smth.png',  dpi=400, bbox_inches='tight')

###### Omega

In [ ]:
# Supp Fig 4 - Omega
test = False 
levels = [np.arange(0,7+0.5,0.25),
         np.arange(-2,3+0.25,0.25)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.Normalize(vmin=0, vmax=7),
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-2, vmax=3)] #vcenter=cbarcent, vmin=minval, vmax=maxval

ds = [dR_add_cyclic_point(var_dR_omega.sel(comb='ik_ic')), 
      dR_add_cyclic_point(var_dR_omega.sel(comb='kp_cm')), 
      dR_add_cyclic_point(var_dR_omega.sel(comb='km_cp')), 
      dR_add_cyclic_point(var_dR_omega.sel(comb='indv-aprx'))]
units_type =  'dR' #or 'ccf'
leg_loc = None#'upper right'
row_titles = [
    r"$\mathbf{Var(dR)}$",
    r"$\mathbf{Var\left(\left(\frac{dR}{dX}\right)'\,\overline{dX}\right)}$",
    r"$\mathbf{Var\left(\overline{\frac{dR}{dX}}\,dX'\right)}$",
    r"$\mathbf{a - (b + c)}$"
]

col_titles = column_titles[:4]

# Plotting Figures =========================================================================================================
row = 4
col = 4

fig = plt.figure(figsize=(24, 14), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.32, hspace=0.27)

add_row_titles_from_gridspec(fig, col_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):#ds
    for j in range(0,col): #exp
        if units_type == 'ccf':
            ylabel = ['$Kday^{-1}K^{-1}$','$K^{-1}$','$hPaDay^{-1}K^{-1}$','$ms^{-1}K^{-1}$']
        elif units_type == 'dR':
            ylabel='$(Wm^{-2}K^{-1})^{2}$'
   
        if test == True: 
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max')#,norm=normalize,levels=levels)
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
            print('test plotting subplot',col*i+j)
        
        elif test == False:     
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max',norm=normalize[0],levels=levels[0])
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[1],levels=levels[1])
            print('plotting subplot',col*i+j)

        make_textbox(ax[i][j],textbox_labels[col*i+j])
        ax[i][j].coastlines()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_global()
        ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
        
        ax[i][j].set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())
        gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        if j == 3:
            gl.left_labels = False
        gl.xlabel_style = {'size': 12}
        gl.ylabel_style = {'size': 12}
        if i == 0:
            ax[i][j].set_title(row_titles[j], fontsize=18,fontweight='bold')

    # var cbar
    cbar_ax = fig.add_axes([0.7, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    cbar = fig.colorbar(map, cax=cbar_ax, orientation='vertical')
    # cbar.set_label(label=ylabel, fontsize=16)
    cbar.ax.tick_params(labelsize=16)

    # covar cbar
    covbar_ax = fig.add_axes([0.91, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    covbar = fig.colorbar(covar_map, cax=covbar_ax, orientation='vertical')
    covbar.set_label(label=ylabel, fontsize=16)
    covbar.ax.tick_params(labelsize=16)
        
plt.savefig('vardR_supp_omega_4x4map_smth.png',  dpi=400, bbox_inches='tight')

###### WS

In [ ]:
# Supp Fig 4 - WS
test = False 
levels = [np.arange(0,6+0.25,0.25),
         np.arange(-2,4+0.25,0.25)] #minval,maxval+cbaritvl,cbaritvl
normalize = [mcolors.Normalize(vmin=0, vmax=6),
            mcolors.TwoSlopeNorm(vcenter=0, vmin=-2, vmax=4)] #vcenter=cbarcent, vmin=minval, vmax=maxval

ds = [dR_add_cyclic_point(var_dR_ws.sel(comb='ik_ic')), 
      dR_add_cyclic_point(var_dR_ws.sel(comb='kp_cm')), 
      dR_add_cyclic_point(var_dR_ws.sel(comb='km_cp')), 
      dR_add_cyclic_point(var_dR_ws.sel(comb='indv-aprx'))]

units_type =  'dR' #or 'ccf'
leg_loc = None#'upper right'
row_titles = [
    r"$\mathbf{Var(dR)}$",
    r"$\mathbf{Var\left(\left(\frac{dR}{dX}\right)'\,\overline{dX}\right)}$",
    r"$\mathbf{Var\left(\overline{\frac{dR}{dX}}\,dX'\right)}$",
    r"$\mathbf{a - (b + c)}$"
]

col_titles = column_titles[:4]

# Plotting Figures =========================================================================================================
row = 4
col = 4

fig = plt.figure(figsize=(24, 14), dpi=250)
ax = [[None for _ in range(col)] for _ in range(row)]
gs = gridspec.GridSpec(row, col, figure=fig, wspace=0.32, hspace=0.27)

add_row_titles_from_gridspec(fig, col_titles, gs, fontsize=18, fontweight='bold')

for i in range(0,row):#ds
    for j in range(0,col): #exp
        if units_type == 'ccf':
            ylabel = ['$Kday^{-1}K^{-1}$','$K^{-1}$','$hPaDay^{-1}K^{-1}$','$ms^{-1}K^{-1}$']
        elif units_type == 'dR':
            ylabel='$(Wm^{-2}K^{-1})^{2}$'
   
        if test == True: 
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max')#,norm=normalize,levels=levels)
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both')#,norm=normalize,levels=levels)
            print('test plotting subplot',col*i+j)
        
        elif test == False:     
            ax[i][j] = fig.add_subplot(gs[i, j], projection=ccrs.PlateCarree(central_longitude=180))
            if j != 3: 
                map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('plasma'),transform=ccrs.PlateCarree(),extend='max',norm=normalize[0],levels=levels[0])
            if j == 3: 
                covar_map = ax[i][j].contourf(ds[j].lon,ds[j].lat,(ds[j].sel(exp=ds[j].exp.values[i])),
                                       cmap=plt.get_cmap('RdBu_r'),transform=ccrs.PlateCarree(),extend='both',norm=normalize[1],levels=levels[1])
            print('plotting subplot',col*i+j)

        make_textbox(ax[i][j],textbox_labels[col*i+j])
        ax[i][j].coastlines()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_global()
        ax[i][j].add_feature(cfeature.LAND, facecolor='lightgray', zorder=3)
        
        ax[i][j].set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())
        gl = ax[i][j].gridlines(draw_labels=True, crs=ccrs.PlateCarree(),linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        if j == 3:
            gl.left_labels = False
        gl.xlabel_style = {'size': 12}
        gl.ylabel_style = {'size': 12}
        if i == 0:
            ax[i][j].set_title(row_titles[j], fontsize=18,fontweight='bold')

    # var cbar
    cbar_ax = fig.add_axes([0.7, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    cbar = fig.colorbar(map, cax=cbar_ax, orientation='vertical')
    # cbar.set_label(label=ylabel, fontsize=16)
    cbar.ax.tick_params(labelsize=16)

    # covar cbar
    covbar_ax = fig.add_axes([0.91, 0.105, 0.008, 0.76])  # [left, bottom, width, height]
    covbar = fig.colorbar(covar_map, cax=covbar_ax, orientation='vertical')
    covbar.set_label(label=ylabel, fontsize=16)
    covbar.ax.tick_params(labelsize=16)
        
plt.savefig('vardR_supp_ws_4x4map_smth.png',  dpi=400, bbox_inches='tight')